In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 문서ID로 직접 상세 크롤링 + 첨부파일 다운로드
# DB 저장 기능 제외 버전

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from pathlib import Path
import mimetypes
import time, re
import traceback
import pandas as pd
import requests
import json

# ─────────────────────────────────────────────────────────
# 설정
# ─────────────────────────────────────────────────────────
BASE = "http://office.anyfive.com/home/"

# ⭐ 크롤링할 문서ID 목록 (여기에 원하는 문서ID 입력)
TARGET_DOC_IDS = [
    "2002390",
    "2008214", 
    "2008497",
]

# BeautifulSoup parser
try:
    import lxml
    BS_PARSER = "lxml"
except:
    BS_PARSER = "html.parser"

# ─────────────────────────────────────────────────────────
# 공통 유틸
# ─────────────────────────────────────────────────────────
driver = None

def wait_click_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))

def wait_css(css, sec=20):
    return WebDriverWait(driver, sec).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))

def click_in_all_frames(css_list=None, xp_list=None, text_contains=None):
    """여러 프레임을 훑으면서 클릭 시도"""
    driver.switch_to.default_content()
    if css_list:
        for css in css_list:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if xp_list:
        for xp in xp_list:
            els = driver.find_elements(By.XPATH, xp)
            if els:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                driver.execute_script("arguments[0].click();", els[0])
                return True
    if text_contains:
        els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
        for el in els:
            if text_contains in (el.text or ""):
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                driver.execute_script("arguments[0].click();", el)
                return True
    frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
    for fr in frames:
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if css_list:
            for css in css_list:
                els = driver.find_elements(By.CSS_SELECTOR, css)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if xp_list:
            for xp in xp_list:
                els = driver.find_elements(By.XPATH, xp)
                if els:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", els[0])
                    driver.execute_script("arguments[0].click();", els[0])
                    return True
        if text_contains:
            els = driver.find_elements(By.XPATH, "//*[self::a or self::button or self::li or self::span]")
            for el in els:
                if text_contains in (el.text or ""):
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    driver.execute_script("arguments[0].click();", el)
                    return True
    driver.switch_to.default_content()
    return False

def switch_into_frame_having(selector_css):
    """selector_css 요소가 보이는 프레임으로 전환"""
    driver.switch_to.default_content()
    if driver.find_elements(By.CSS_SELECTOR, selector_css):
        return True
    for fr in driver.find_elements(By.CSS_SELECTOR, "iframe, frame"):
        driver.switch_to.default_content()
        driver.switch_to.frame(fr)
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            return True
    driver.switch_to.default_content()
    return False

def norm_space(s: str) -> str:
    if not s: return ""
    s = s.replace("\xa0", " ")
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def dedupe_attachments(items):
    seen, out = set(), []
    for it in items or []:
        name = (it.get("파일명") or "").strip()
        href = (it.get("href") or "").strip()
        key = (name, href)
        if not name and not href:
            continue
        if key in seen:
            continue
        seen.add(key)
        clean = {k: v for k, v in it.items() if v not in (None, "", [])}
        out.append(clean)
    return out

# ─────────────────────────────────────────────────────────
# 파일 확장자 감지
# ─────────────────────────────────────────────────────────
def get_file_extension(url, content_type=None, content_bytes=None, default_ext=".bin"):
    """URL과 Content-Type에서 올바른 확장자 추출"""
    if content_bytes:
        if content_bytes[:2] == b'\xff\xd8':
            return '.jpg'
        elif content_bytes[:8] == b'\x89PNG\r\n\x1a\n':
            return '.png'
        elif content_bytes[:6] in (b'GIF87a', b'GIF89a'):
            return '.gif'
        elif content_bytes[:4] == b'RIFF' and content_bytes[8:12] == b'WEBP':
            return '.webp'
        elif content_bytes[:2] == b'BM':
            return '.bmp'
    
    if content_type:
        ct = content_type.split(';')[0].strip().lower()
        if 'jpeg' in ct or 'jpg' in ct:
            return '.jpg'
        if 'png' in ct:
            return '.png'
        if 'gif' in ct:
            return '.gif'
        if 'webp' in ct:
            return '.webp'
        ext = mimetypes.guess_extension(ct)
        if ext:
            return '.jpg' if ext == '.jpe' else ext
    
    path = urlparse(url).path
    if '.' in path.split('/')[-1]:
        ext = Path(path).suffix.lower()
        if ext in ['.jpg', '.jpeg', '.png', '.gif', '.webp', '.bmp', '.svg',
                   '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.zip', '.txt']:
            return ext
    
    return default_ext

def sanitize_filename(filename):
    """파일명에서 위험한 문자 제거"""
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
    if len(filename) > 200:
        name, ext = Path(filename).stem, Path(filename).suffix
        filename = name[:190] + ext
    return filename

# ─────────────────────────────────────────────────────────
# 상세 페이지 파싱 함수들
# ─────────────────────────────────────────────────────────
def make_session_from_driver(drv, base_url):
    """Selenium 쿠키를 requests Session으로 변환"""
    s = requests.Session()
    host = urlparse(base_url).hostname
    for c in drv.get_cookies():
        s.cookies.set(c["name"], c["value"], domain=c.get("domain") or host)
    s.headers.update({"X-Requested-With": "XMLHttpRequest"})
    return s

def parse_kv_tables(drv):
    """상세 페이지 상단 Key-Value 테이블 파싱"""
    soup = BeautifulSoup(drv.page_source, BS_PARSER)
    out = {}
    
    title = soup.select_one(".apr_title")
    if title:
        out["제목"] = title.get_text(" ", strip=True)
    
    for tbl in soup.select("table.tbl_none-titInfo"):
        for tr in tbl.select("tr"):
            ths = [th.get_text(" ", strip=True) for th in tr.select("th")]
            tds = [td.get_text("\n", strip=True) for td in tr.select("td")]
            for i in range(min(len(ths), len(tds))):
                key = ths[i].replace(" ", "")
                val = re.sub(r"\n+", "\n", tds[i]).strip()
                if "기안일완료일" in key or ("기안일" in key and "완료" in key):
                    dts = re.findall(r"\d{4}[-/]\d{2}[-/]\d{2}(?:\s+\d{2}:\d{2}:\d{2})?", val)
                    if len(dts) >= 1:
                        out["기안일"] = dts[0].replace("-", "/")
                    if len(dts) >= 2:
                        out["완료일"] = dts[1].replace("-", "/")
                else:
                    out[key] = val
    
    attach = []
    for a in soup.select("#aprShowFileMap a"):
        attach.append({
            "파일명": a.get_text(" ", strip=True),
            "href": urljoin(BASE, a.get("href", "")),
        })
    if attach:
        out["_첨부파일_anchor"] = attach
    
    return out

def download_attachments(attachments, session, base_url, doc_id, out_dir="detailed_docs/attachments"):
    """첨부파일 실제 다운로드"""
    if not attachments:
        print(f"  ℹ️  첨부파일 없음")
        return []
    
    att_dir = Path(out_dir) / str(doc_id)
    att_dir.mkdir(parents=True, exist_ok=True)
    
    results = []
    success_count = 0
    
    for i, att in enumerate(attachments, 1):
        href = att.get("href", "")
        if not href:
            continue
        
        filename = att.get("파일명", f"attachment_{i}")
        abs_url = urljoin(base_url, href)
        
        try:
            print(f"  📥 [{i}/{len(attachments)}] 다운로드 중: {filename}")
            r = session.get(abs_url, timeout=30)
            r.raise_for_status()
            
            content_type = r.headers.get('Content-Type', '')
            ext = get_file_extension(abs_url, content_type, content_bytes=r.content)
            
            safe_name = sanitize_filename(filename)
            if not Path(safe_name).suffix:
                safe_name = safe_name + ext
            
            final_name = f"{int(time.time()*1000)}_{safe_name}"
            file_path = att_dir / final_name
            file_path.write_bytes(r.content)
            
            results.append({
                **att,
                "saved_as": str(file_path),
                "saved_filename": final_name,
                "size": len(r.content),
                "content_type": content_type
            })
            success_count += 1
            print(f"      ✅ 저장: {file_path.name} ({len(r.content):,} bytes)")
            
        except Exception as e:
            results.append({**att, "error": str(e)})
            print(f"      ❌ 실패: {filename} - {e}")
    
    if results:
        manifest = {
            "문서ID": doc_id,
            "총파일수": len(attachments),
            "성공": success_count,
            "실패": len(attachments) - success_count,
            "파일목록": results
        }
        manifest_path = att_dir / "_manifest.json"
        manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
    
    return results

def extract_body_bundle(drv, base_url, doc_id, out_dir, session):
    """본문 HTML/텍스트/표/이미지 추출"""
    body_html = drv.execute_script("""
      const pick = () => {
        const sel = ['#appr_doc_cont', '#apprDocContent', '#tempApprDocContent', '.content_box'];
        for (const s of sel) {
          const el = document.querySelector(s);
          if (el) return el.innerHTML;
        }
        return '';
      };
      return pick();
    """) or ""
    
    if not body_html.strip():
        for f in drv.find_elements(By.CSS_SELECTOR, "iframe[id^='editorIframe_approvalShow_']"):
            drv.switch_to.frame(f)
            body_html = drv.execute_script("return document.body ? document.body.innerHTML : '';") or ""
            drv.switch_to.default_content()
            break
    
    bundle = {"body_html": body_html, "body_text": "", "body_tables": [], "body_images": []}
    
    if body_html.strip():
        soup = BeautifulSoup(body_html, BS_PARSER)
        for t in soup(["script", "style"]):
            t.decompose()
        txt = soup.get_text("\n")
        bundle["body_text"] = re.sub(r"\n{3,}", "\n\n", txt).strip()
        
        try:
            from io import StringIO
            dfs = pd.read_html(StringIO(body_html))
            for df in dfs:
                df.columns = [str(c).strip() for c in df.columns]
                bundle["body_tables"].append(df.fillna("").to_dict(orient="records"))
        except:
            pass
        
        images = soup.select("img[src]")
        if images:
            img_dir = Path(out_dir) / "images" / str(doc_id)
            img_dir.mkdir(parents=True, exist_ok=True)
            
            for i, img in enumerate(images, 1):
                src = img.get("src", "").strip()
                if not src:
                    continue
                abs_url = urljoin(base_url, src)
                
                try:
                    r = session.get(abs_url, timeout=15)
                    r.raise_for_status()
                    
                    content_type = r.headers.get('Content-Type', '')
                    ext = get_file_extension(abs_url, content_type, content_bytes=r.content, default_ext=".png")
                    
                    fn = f"img_{int(time.time()*1000)}_{i}{ext}"
                    p = img_dir / fn
                    p.write_bytes(r.content)
                    
                    bundle["body_images"].append({
                        "src": abs_url,
                        "saved_as": str(p),
                        "size": len(r.content)
                    })
                except Exception as e:
                    bundle["body_images"].append({"src": abs_url, "error": str(e)})
    
    return bundle

def parse_approval_line_table(drv):
    """결재선 테이블 파싱"""
    soup = BeautifulSoup(drv.page_source, BS_PARSER)
    out = []
    for tr in soup.select("#apprLineTable tbody tr"):
        tds = [td.get_text(" ", strip=True) for td in tr.select("td")]
        if len(tds) >= 6:
            out.append({
                "순서": tds[0], "구분": tds[1], "상태": tds[2],
                "결재일시": tds[3], "결재부서": tds[4], "결재자": tds[5]
            })
    return out

def parse_approval_opinions(drv):
    """결재의견 파싱"""
    soup = BeautifulSoup(drv.page_source, BS_PARSER)
    results = []
    
    ul_list = soup.select('table.tbl_none-titInfo ul._mp, table.tbl_none-titInfo ul._np')
    for ul in ul_list:
        items = ul.find_all('li')
        if len(items) < 2:
            continue
        head = items[0]
        body = items[1]
        
        code_el = head.select_one('[class^=approvalOpinionCode]') or head.find('span')
        code = code_el.get_text(strip=True) if code_el else ''
        
        user_a = head.select_one('span.F_12_black_b a.abcUsr')
        user = user_a.get_text(strip=True) if user_a else ''
        
        dt = ''
        for sp in head.find_all('span'):
            t = sp.get_text(strip=True)
            if t and t[0].isdigit() and ('-' in t or '/' in t) and ':' in t:
                dt = t
        
        msg = body.get_text(" ", strip=True)
        results.append({
            "단계": code,
            "작성자": user,
            "시간": dt,
            "내용": msg
        })
    
    return results

def fetch_comments(session, base_url, apr_doc_seq):
    """댓글 XHR"""
    try:
        url = urljoin(base_url, "/apr/doc/aprCmtlist.jstl")
        r = session.get(url, params={"aprDocSeq": apr_doc_seq}, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, BS_PARSER)
        out = []
        for ul in soup.select("ul._mp"):
            head = ul.select_one("li > span.F_12_black_b")
            code = ul.select_one("li > span[class^=approvalOpinionCode]")
            time_el = ul.select_one("li > span[style*='margin-left']")
            body = ul.select_one("li + li")
            out.append({
                "단계": code.get_text(strip=True) if code else "",
                "작성자": head.get_text(" ", strip=True) if head else "",
                "시간": time_el.get_text(strip=True) if time_el else "",
                "내용": body.get_text("\n", strip=True) if body else ""
            })
        return out
    except:
        return []

def scrape_detailed_doc(drv, doc_id, base_url=BASE, out_dir="detailed_docs"):
    """상세 페이지에서 모든 데이터 수집"""
    WebDriverWait(drv, 15).until(lambda d: d.execute_script("return document.readyState") == "complete")
    
    try:
        WebDriverWait(drv, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".apr_title")))
    except:
        print(f"  ⚠️ 제목 요소 대기 실패, 계속 진행...")
    
    apr_doc_seq = drv.execute_script("var el=document.querySelector('input#appr_doc_seq');return el?el.value:'';")
    if not apr_doc_seq:
        apr_doc_seq = doc_id
    
    session = make_session_from_driver(drv, base_url)
    
    # 상단 정보
    kv = parse_kv_tables(drv)
    
    # 첨부파일 수집 및 다운로드
    print(f"\n  📎 첨부파일 다운로드 시작...")
    attach_from_dom = []
    soup = BeautifulSoup(drv.page_source, BS_PARSER)
    for a in soup.select("#aprShowFileMap a"):
        attach_from_dom.append({
            "파일명": a.get_text(" ", strip=True),
            "href": urljoin(base_url, a.get("href", "")),
        })
    
    anchor_from_kv = kv.get("_첨부파일_anchor") or []
    attach_list = dedupe_attachments(anchor_from_kv + attach_from_dom)
    attach = download_attachments(attach_list, session, base_url, apr_doc_seq, out_dir + "/attachments")
    
    # 본문
    print(f"\n  📄 본문 추출 시작...")
    body_bundle = extract_body_bundle(drv, base_url, apr_doc_seq, out_dir, session)
    
    # 댓글
    comments = fetch_comments(session, base_url, apr_doc_seq)
    
    # 결재선
    appr_grid = parse_approval_line_table(drv)
    
    # 결재의견
    approval_opinions = parse_approval_opinions(drv)
    
    # 최종 문서 구성
    doc = {
        "문서ID": doc_id,
        "appr_doc_seq": apr_doc_seq,
        "상단정보": kv,
        "첨부파일": attach,
        "결재선(표)": appr_grid,
        "본문": body_bundle,
        "댓글": comments,
        "결재의견": approval_opinions,
    }
    
    # JSON 저장
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)
    json_path = out_path / f"doc_{doc_id}.json"
    json_path.write_text(json.dumps(doc, ensure_ascii=False, indent=2), encoding="utf-8")
    print(f"\n  ✅ JSON 저장: {json_path}")
    
    return doc

def normalize_for_export(doc):
    """DataFrame용 정규화"""
    top = doc.get("상단정보", {}) or {}
    body = doc.get("본문", {}) or {}
    
    attach = dedupe_attachments((top.get("_첨부파일_anchor") or []) + (doc.get("첨부파일") or []))
    
    content = {
        "상단정보": top,
        "본문": body,
        "첨부파일": attach,
        "결재선_표": doc.get("결재선(표)") or [],
        "결재의견": doc.get("결재의견") or [],
        "댓글": doc.get("댓글") or [],
    }
    
    row = {
        "문서ID": doc.get("문서ID") or doc.get("appr_doc_seq") or "",
        "제목": top.get("제목") or "",
        "양식명": top.get("양식명", ""),
        "문서번호": top.get("문서번호", ""),
        "기안자": top.get("기안자", ""),
        "기안일": top.get("기안일", ""),
        "상태": top.get("상태", ""),
        "내용": json.dumps(content, ensure_ascii=False)
    }
    return row

def back_to_list():
    """상세에서 목록으로 복귀"""
    try:
        driver.switch_to.default_content()
        switch_into_frame_having(".tabConts_headerBx, #apprDocContent")
        
        selectors = [
            "button.k-button[onclick*='onClick_findFormBtn']",
            "button[onclick*='onClick_findFormBtn']",
            "button[title='목록']",
        ]
        for css in selectors:
            els = driver.find_elements(By.CSS_SELECTOR, css)
            if els:
                driver.execute_script("arguments[0].click();", els[0])
                time.sleep(1.5)
                return True
        
        driver.back()
        time.sleep(1.5)
        return True
    except:
        return False

# ─────────────────────────────────────────────────────────
# 메인 실행
# ─────────────────────────────────────────────────────────
def main():
    global driver
    
    print("="*70)
    print("📋 문서ID 기반 상세 크롤링 시작")
    print(f"   대상 문서: {len(TARGET_DOC_IDS)}건")
    print(f"   문서ID: {TARGET_DOC_IDS}")
    print("="*70)
    
    # 브라우저 설정
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-popup-blocking")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option('useAutomationExtension', False)
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(service=Service(), options=opts)
    driver.get(BASE)
    time.sleep(1.2)
    
    input("\n🔐 로그인 완료 후 Enter를 눌러주세요... ")
    
    # 전자결재 → 결재 문서관리 메뉴 진입
    print("\n📁 전자결재 메뉴 진입 중...")
    click_in_all_frames(
        css_list=["a[href='/apr/'].left_menu", "a[href='/apr/']"],
        xp_list=["//a[@href='/apr/']"],
        text_contains="전자결재"
    )
    time.sleep(0.8)
    
    click_in_all_frames(
        css_list=["li[onclick*='managementDoc']"],
        xp_list=["//*[@onclick and contains(.,'managementDoc')]"],
        text_contains="결재 문서관리"
    )
    time.sleep(1.5)
    
    # 프레임 전환
    switch_into_frame_having("header, .tabConts_headerBx")
    
    # 결과 저장용
    details = []
    attachments_rows = []
    success_count = 0
    error_count = 0
    
    # 각 문서ID에 대해 크롤링
    for idx, doc_id in enumerate(TARGET_DOC_IDS):
        print(f"\n{'='*70}")
        print(f"[{idx+1}/{len(TARGET_DOC_IDS)}] 문서ID: {doc_id} 처리 중...")
        print("="*70)
        
        try:
            # 프레임 확인
            driver.switch_to.default_content()
            switch_into_frame_having("header, .tabConts_headerBx, #apprDocContent")
            time.sleep(0.5)
            
            # ⭐ 문서ID로 직접 상세 열기 (핵심!)
            print(f"  📄 상세 페이지 열기...")
            driver.execute_script(f"managementDocList.clickGridRow({doc_id});")
            time.sleep(2.5)
            
            # 상세 페이지 로드 대기
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#apprLineTable, .apr_title"))
                )
            except:
                print(f"  ⚠️ 상세 페이지 로드 대기 실패")
            
            time.sleep(1)
            
            # 상세 수집
            doc_json = scrape_detailed_doc(driver, doc_id, base_url=BASE, out_dir="detailed_docs")
            
            # 정규화
            flat = normalize_for_export(doc_json)
            details.append(flat)
            
            # 첨부파일 정보 추출
            for att in doc_json.get("첨부파일", []):
                att_row = {
                    "문서ID": doc_id,
                    "파일이름": att.get("파일명", ""),
                    "경로": att.get("href", ""),
                    "저장경로": att.get("saved_as", ""),
                    "파일크기": att.get("size", 0),
                    "content_type": att.get("content_type", "")
                }
                attachments_rows.append(att_row)
            
            success_count += 1
            print(f"  ✅ 수집 완료: {flat.get('제목', '')[:50]}")
            
            # 목록으로 복귀
            back_to_list()
            time.sleep(1)
            
        except Exception as e:
            error_count += 1
            print(f"  ❌ 오류: {str(e)}")
            traceback.print_exc()
            
            try:
                back_to_list()
                time.sleep(1)
            except:
                pass
    
    # ─────────────────────────────────────────────────────────
    # 결과 저장
    # ─────────────────────────────────────────────────────────
    print("\n" + "="*70)
    print("📊 크롤링 완료!")
    print("="*70)
    print(f"  ✅ 성공: {success_count}건")
    print(f"  ❌ 실패: {error_count}건")
    
    if details:
        # Excel/CSV 저장
        cols = ["문서ID", "제목", "양식명", "문서번호", "기안자", "기안일", "상태", "내용"]
        df = pd.DataFrame(details, columns=cols)
        
        for c in df.select_dtypes(include=["object"]).columns:
            df[c] = df[c].map(lambda x: norm_space(x) if isinstance(x, str) else x)
        
        df.to_excel("전자결재_목록.xlsx", index=False, engine="openpyxl")
        df.to_csv("전자결재_목록.csv", index=False, encoding="utf-8-sig")
        print(f"\n📁 저장 완료:")
        print(f"   - 전자결재_목록.xlsx")
        print(f"   - 전자결재_목록.csv")
        
        # 첨부파일 목록 저장
        if attachments_rows:
            att_df = pd.DataFrame(attachments_rows)
            att_df.to_excel("전자결재_첨부파일.xlsx", index=False, engine="openpyxl")
            att_df.to_csv("전자결재_첨부파일.csv", index=False, encoding="utf-8-sig")
            print(f"   - 전자결재_첨부파일.xlsx ({len(attachments_rows)}건)")
            print(f"   - 전자결재_첨부파일.csv")
    
    print(f"\n📂 상세 JSON: detailed_docs/ 폴더")
    print(f"📂 첨부파일: detailed_docs/attachments/[문서ID]/")
    print(f"📂 본문이미지: detailed_docs/images/[문서ID]/")
    print("="*70)
    
    # driver.quit()

if __name__ == "__main__":
    main()

📋 문서ID 기반 상세 크롤링 시작
   대상 문서: 3건
   문서ID: ['2002390', '2008214', '2008497']



🔐 로그인 완료 후 Enter를 눌러주세요...  



📁 전자결재 메뉴 진입 중...

[1/3] 문서ID: 2002390 처리 중...
  📄 상세 페이지 열기...

  📎 첨부파일 다운로드 시작...
  📥 [1/1] 다운로드 중: 카드사용실적보고(20100527)_편영수K.xls (96.50KB)
      ✅ 저장: 1764220907894_카드사용실적보고(20100527)_편영수K.xls (96.50KB) (98,816 bytes)

  📄 본문 추출 시작...

  ✅ JSON 저장: detailed_docs\doc_2002390.json
  ✅ 수집 완료: 법인카드 사용내역 정산 보고_5월_편영수K

[2/3] 문서ID: 2008214 처리 중...
  📄 상세 페이지 열기...

  📎 첨부파일 다운로드 시작...
  📥 [1/1] 다운로드 중: 법인카드_사용내역보고_송준선(201501).xlsx (91.81KB)
      ✅ 저장: 1764220915363_법인카드_사용내역보고_송준선(201501).xlsx (91.81KB) (94,015 bytes)

  📄 본문 추출 시작...

  ✅ JSON 저장: detailed_docs\doc_2008214.json
  ✅ 수집 완료: 2015년 1월 CARD 사용내역 정산 보고	2015년01월 법인카드 정산내역 품의(송준선

[3/3] 문서ID: 2008497 처리 중...
  📄 상세 페이지 열기...

  📎 첨부파일 다운로드 시작...
  📥 [1/3] 다운로드 중: 20150330_보안서약서_박석은.doc (34.00KB)
      ✅ 저장: 1764220922670_20150330_보안서약서_박석은.doc (34.00KB) (34,816 bytes)
  📥 [2/3] 다운로드 중: 20150330_토피아정보기술_LGD_개발용역계약서_박석은.doc (72.00KB)
      ✅ 저장: 1764220922706_20150330_토피아정보기술_LGD_개발용역계약서_박석은.doc (72.00KB) (73,728 bytes)
  📥 [3/3]